In [1]:
import ffmpeg
import os
import cv2

def video2img(dir_video, dir_imgs):
    stream = ffmpeg.input(dir_video)
    stream = ffmpeg.output(stream, dir_imgs)
    ffmpeg.run(stream)

def imgs2video(imgs_dir, video_dir):
    stream = ffmpeg.input(imgs_dir)
    stream = ffmpeg.output(stream, video_dir, pix_fmt='yuv420p')
    ffmpeg.run(stream)

In [6]:
input_dir = "./youku/LOW/"
output_dir = "./youku/IMG_LOW/"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
list_files = os.listdir(input_dir)

for i in range(1):
    file_name = list_files[i]
    dir_video = input_dir + file_name
    dir_out = output_dir + file_name[:-4] + '/'
    if not os.path.exists(dir_out):
        os.mkdir(dir_out)
    dir_out = dir_out + "%3d.bmp"
    video2img(dir_video, dir_out)

In [10]:
img_path = os.path.join(output_dir, 'Youku_00000_l', '001.bmp')
img = cv2.imread(img_path)
cv2.imshow(img_path, img)
cv2.waitKey(0)


-1

In [15]:
# 传统插值算法生成图片
img_path = os.path.join('youku', 'IMG_LOW', 'Youku_00000_l', '034.bmp')
img_l = cv2.imread(img_path)
img_path = os.path.join('youku', 'IMG_HIGH', 'Youku_00000_h_GT', '034.bmp')
img_h = cv2.imread(img_path)

size_l = (480, 270)
size_h = (1920, 1080)
new_img = cv2.resize(img_l, size_h, interpolation=cv2.INTER_NEAREST)
cv2.imwrite("youku/GEN/034_INTER_NEAREST.bmp", new_img)

new_img = cv2.resize(img_l, size_h, interpolation=cv2.INTER_LINEAR)
cv2.imwrite("youku/GEN/034_INTER_LINEAR.bmp", new_img)

new_img = cv2.resize(img_l, size_h, interpolation=cv2.INTER_CUBIC)
cv2.imwrite("youku/GEN/034_INTER_CUBIC.bmp", new_img)

True

In [17]:
def psnr(img1, img2):
    import numpy as np
    import math
    mse = np.mean((img1 - img2) ** 2)
    if mse <= 1.0e-10:
        return 100
    return 10 * math.log10(255.0 ** 2/ mse)

path = "youku/GEN/034_INTER_NEAREST.bmp"
img_INTER_NEAREST = cv2.imread(path)
print("INTER NEAREST:", psnr(img_h, img_INTER_NEAREST))

path = "youku/GEN/034_INTER_LINEAR.bmp"
img_INTER_LINEAR = cv2.imread(path)
print("INTER LINEAR:", psnr(img_h, img_INTER_LINEAR))

path = "youku/GEN/034_INTER_CUBIC.bmp"
img_INTER_CUBIC = cv2.imread(path)
print("INTER CUBIC:", psnr(img_h, img_INTER_CUBIC))

INTER NEAREST: 38.39652116975954
INTER LINEAR: 38.89646804712031
INTER CUBIC: 38.92935344332646


In [22]:
# Bicubic
def base_function(x, a=-0.5):
    import numpy as np
    Wx = 0
    if np.abs(x) <= 1:
        Wx = (a + 2) * (np.abs(x) ** 3) - (a + 3)*x**2 + 1
    elif 1 <= np.abs(x) <= 2:
        Wx = a * (np.abs(x)**3) - 5 * a * (np.abs(x)**2) + 8 * a * np.abs(x) - 4 * a
    return Wx

def padding(img):
    import numpy as np
    h, w, c = img.shape
    print(img.shape)
    pad_image = np.zeros((h+4, w+4, c))
    pad_image[2:h+2, 2:w+2] = img
    return pad_image

def bicubic(img, scale, a=-0.5):
    import numpy as np
    print("Bicubic")
    h, w, color =img.shape
    img = padding(img)
    nh = h * scale
    nw = w * scale
    new_img = np.zeros((nh, nw, color))

    for c in range(color):
        for i in range(nw):
            for j in range(nh):
                px = i / scale + 2
                py = j / scale + 2
                px_int = int(px)
                py_int = int(py)
                u = px - px_int
                v = py - py_int

                A = np.matrix([[base_function(u + 1, a)], [base_function(u, a)], [base_function(u - 1, a)], [base_function(u- 2, a)]])
                C = np.matrix([base_function(v+1, a), base_function(v, a), base_function(v-1, a), base_function(v-2, a)])
                B = np.matrix([[img[py_int-1, px_int-1][c], img[py_int-1, px_int][c],
                                img[py_int-1, px_int+1][c], img[py_int-1, px_int+2][c]],
                               [img[py_int, px_int-1][c], img[py_int, px_int][c],
                                img[py_int, px_int+1][c], img[py_int, px_int+2][c]],
                               [img[py_int+1, px_int-1][c], img[py_int+1, px_int][c],
                                img[py_int+1, px_int+1][c], img[py_int+1, px_int+2][c]],
                               [img[py_int+2, px_int-1][c], img[py_int+2, px_int][c],
                                img[py_int+2, px_int+1][c], img[py_int+2, px_int+2][c]]])
                new_img[j, i][c] = np.dot(np.dot(C, B), A)

    return new_img

scale = 4
new_img = bicubic(img_l, scale)
cv2.imwrite("youku/GEN/034_l_bicubic.png", new_img)
print("Finish")

Bicubic
(270, 480, 3)
Finish


In [21]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [29]:
interpolation = cv2.INTER_CUBIC
def srcnn():
    # bicubic
    inputs = keras.layers.Input(shape=(1080, 1920, 3))
    # CNN 9 * 9 * 64 padding=same
    cnn = keras.layers.Conv2D(64, 9, padding='same', activation='relu')(inputs)
    # kernel 1 * 1 * 32
    cnn = keras.layers.Conv2D(32, 1, padding='same', activation='relu')(cnn)
    # CNN 5 * 5 * 3
    outputs = keras.layers.Conv2D(3, 5, padding='same')(inputs)

    model = keras.models.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=tf.optimizers.Adam(1e-1), loss=tf.losses.mse, metrics=['mse'])
    return model

model = srcnn()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1080, 1920, 3)]   0         
                                                                 
 conv2d_5 (Conv2D)           (None, 1080, 1920, 3)     228       
                                                                 
Total params: 228
Trainable params: 228
Non-trainable params: 0
_________________________________________________________________


In [32]:
plateau = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', verbose=0, mode='min', factor=0.1, patience=6)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', verbose=0, mode='min', patience=25)
checkpoint = keras.callbacks.ModelCheckpoint('srcnn.h5', monitor='val_loss', verbose=0, mode='min', save_best_only=True)

img_bicubic = cv2.resize(img_l, (1920, 1080), interpolation=cv2.INTER_CUBIC)
X = np.array([img_INTER_CUBIC, img_INTER_CUBIC])
y = np.array([img_h, img_h])

model.fit(X, y, epochs=10, batch_size=2, verbose=1, shuffle=True, validation_data=(X, y), callbacks=[plateau, early_stopping, checkpoint])
model.evaluate(X, y, verbose=0)

pic_super = model.predict(X, verbose=0, batch_size=1)
cv2.imwrite("youku/GEN/0034_scrnn.bmp", pic_super[0])


Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 41998.7539 - mse: 41998.7539 - val_loss: 38967.1328 - val_mse: 38967.1328 - lr: 0.0100
Epoch 2/10
1/1 [==============================] - 2s 2s/step - loss: 38967.1328 - mse: 38967.1328 - val_loss: 34159.4805 - val_mse: 34159.4805 - lr: 0.0100
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 34159.4805 - mse: 34159.4805 - val_loss: 28247.6641 - val_mse: 28247.6641 - lr: 0.0100
Epoch 4/10
1/1 [==============================] - 2s 2s/step - loss: 28247.6641 - mse: 28247.6641 - val_loss: 21947.4160 - val_mse: 21947.4160 - lr: 0.0100
Epoch 5/10
1/1 [==============================] - 2s 2s/step - loss: 21947.4160 - mse: 21947.4160 - val_loss: 15923.5273 - val_mse: 15923.5273 - lr: 0.0100
Epoch 6/10
1/1 [==============================] - 2s 2s/step - loss: 15923.5273 - mse: 15923.5273 - val_loss: 

True